In [1]:
import os
import numpy as np
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [2]:
# show all columns
pd.options.display.max_columns = None

## Join Bike volume data to links

In [3]:
# read in links csv
links = pd.read_csv(r".\Model_Outputs\links.csv")

# read in links shapefile
links_shp = pd.DataFrame.spatial.from_featureclass(r".\Model_Outputs\links.shp")
print(links.shape)
print(links_shp.shape)


(142104, 33)
(142104, 39)


In [4]:
# read in bike volume
bike_volume = pd.read_csv(r".\Model_Outputs\bike_vol.csv")

#should be double the amount of links for both directions
print(bike_volume.shape)

# fill bike volume NAs with -1
bike_volume['bike_vol'] = bike_volume['bike_vol'].fillna(0)


# REMOVE THIS WHEN TABLE BUG IS FIXED
bike_volume.dropna(inplace=True)

# convert node ids to int
bike_volume['from_node'] = bike_volume['from_node'].astype(int)
bike_volume['to_node'] = bike_volume['to_node'].astype(int)

(548102, 3)


In [5]:
# Create key to use for joining to links
bike_volume['key'] = np.where(bike_volume['from_node'] < bike_volume['to_node'], 
                              bike_volume['from_node'].astype(str) + "_"+ bike_volume['to_node'].astype(str), 
                              bike_volume['to_node'].astype(str) + "_"+ bike_volume['from_node'].astype(str))

# Create directional keys
bike_volume['ft_key'] = bike_volume['from_node'].astype(str) + "_"+ bike_volume['to_node'].astype(str)
bike_volume['tf_key'] = bike_volume['to_node'].astype(str) + "_"+ bike_volume['from_node'].astype(str)

bike_volume.head(15)

,from_node,to_node,bike_vol,key,ft_key,tf_key
0,20,18,10.299702,18_20,20_18,18_20
1,27,28,0.861175,27_28,27_28,28_27
2,50,27,0.861175,27_50,50_27,27_50
3,135,911,14.950139,135_911,135_911,911_135
4,405,868,0.032578,405_868,405_868,868_405
5,431,429,59.211980,429_431,431_429,429_431
6,467,474,47.462292,467_474,467_474,474_467
7,491,533,29.119989,491_533,491_533,533_491
8,512,526,4.434876,512_526,512_526,526_512
9,526,558,4.434876,526_558,526_558,558_526


In [6]:
# summarize trips in each direction
ft_vol_sum = pd.DataFrame(bike_volume.groupby('ft_key')['bike_vol'].sum())
tf_vol_sum = pd.DataFrame(bike_volume.groupby('tf_key')['bike_vol'].sum())

ft_vol_sum.columns = ['ft_bvol']
tf_vol_sum.columns = ['tf_bvol']

tf_vol_sum.head()

,tf_bvol
tf_key,
0_0,9.310895e+06
100002_97471,6.104504e+02
100007_97441,1.966764e+00
100009_97399,1.785114e+02
100009_97419,1.578244e-01


In [7]:
# summarize trips in both directions
volume_sum = pd.DataFrame(bike_volume.groupby('key')['bike_vol'].sum())
volume_sum.columns = ['total_bvol']
volume_sum.head(10)

,total_bvol
key,
0_0,9.310895e+06
10002_10036,1.443086e-01
10003_10008,9.754950e+00
10008_10026,5.425563e+01
10010_10025,5.502669e+00
10010_10027,1.211232e+01
10016_10018,2.126788e-01
10017_10020,8.703915e+00
10017_10024,1.908585e+00


In [8]:
#Create FTkey and TF key to use for joining to bike volumes
links['key'] = np.where(links['from_node'].astype(int) < links['to_node'].astype(int), 
                              links['from_node'].astype(str) + "_"+ links['to_node'].astype(str), 
                              links['to_node'].astype(str) + "_"+ links['from_node'].astype(str))

links[['from_node', 'to_node','key']].head(10)

,from_node,to_node,key
0,3,935,3_935
1,4,934,4_934
2,4,3,3_4
3,5,4,4_5
4,6,5,5_6
5,7,6,6_7
6,8,7,7_8
7,9,5,5_9
8,10,9,9_10
9,10,6,6_10


In [9]:
# copy the links table
links2 = links[['link_id', 'key']].copy()

# join the links with the bike volumes using the common keys
link_bike_vol = links2.merge(volume_sum, left_on='key', right_on='key', how='left')
link_bike_vol2 = link_bike_vol.merge(ft_vol_sum, left_on='key', right_on='ft_key', how='left')
link_bike_vol3 = link_bike_vol2.merge(tf_vol_sum, left_on='key', right_on='tf_key', how='left')

# examine the results
print(links2.shape)
print(link_bike_vol3.shape)

(142104, 2)
(142104, 5)


In [10]:
link_bike_vol3.head(5)

,link_id,key,total_bvol,ft_bvol,tf_bvol
0,0,3_935,NaN,NaN,NaN
1,1,4_934,NaN,NaN,NaN
2,2,3_4,11.905264,1.170316,10.734948
3,3,4_5,3.444762,1.804172,1.640590
4,4,5_6,3.444762,1.804172,1.640590


In [11]:
# Examine the column names
links_shp.columns

Index(['FID', 'Join_Count', 'TARGET_FID', 'FID_lines_', 'Name', 'Oneway',
       'Speed', 'AutoNetwor', 'BikeNetwor', 'PedNetwork', 'SourceData',
       'DriveTime', 'BikeTime', 'Pedestrian', 'Length_Mil', 'ConnectorN',
       'RoadClass', 'AADT', 'AADT_YR', 'BIKE_L', 'BIKE_R', 'Shape_Leng', 'id',
       'Start_Key', 'End_Key', 'Bike_Lane', 'Bike_Path', 'Bike_Blvd', 'SIGID',
       'Signal', 'BUFF_DIST', 'ORIG_FID', 'link_id', 'from_z', 'to_z',
       'Slope_AB', 'Slope_BA', 'Slope_Per', 'SHAPE'],
      dtype='object')

In [12]:
# export final result to csv
link_bike_vol3['link_id'] = link_bike_vol3['link_id'].astype('int64')

# join bike vol to links shapefile
links4 = links_shp.merge(link_bike_vol3, left_on='link_id', right_on='link_id', how='left')

links4.fillna(0, inplace=True)


# export to shape
links4.spatial.to_featureclass(location=r".\Default.gdb\links_bv")


'E:\\Projects\\igraph\\Post_Process_Bike_Model_Outputs\\Default.gdb\\links_bv'

## Summarize zone trips by Attracting/Producing Zone

In [13]:
# read in zones
zones = pd.DataFrame.spatial.from_featureclass(r".\Model_Outputs\microzones.shp")
zones.head()

,FID,Id,zone_id,CO_TAZID,TAZID,CO_FIPS,CO_NAME,residentia,households,population,jobs1,jobs3,jobs4,jobs5,jobs6,jobs7,jobs9,jobs10,jobs_total,AVGINCOME,ENROL_ELEM,ENROL_MIDL,ENROL_HIGH,HHSIZE_LC1,HHSIZE_LC2,HHSIZE_LC3,PCT_POPLC1,PCT_POPLC2,PCT_POPLC3,PCT_AG1,PCT_AG2,PCT_AG3,INC1,INC2,INC3,INC4,PARK_SCORE,PARK_AREA,SCHOOL_CD,COLL_ENROL,TH_SCORE,COMM_RAIL,LIGHT_RAIL,GQU_RATIO,MIXED_USE,bike_share,industrial,NODE_ID,bklane_len,bkpath_len,AREA_SQMIL,SHAPE
0,0,0,0,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0.0,0.0,0,0,75772,0.753915,0.0,0.037866,"{""rings"": [[[432271.91339999996, 4511646.3509]..."
1,1,0,1,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0.0,0.0,0,0,76508,2.480609,0.0,0.637239,"{""rings"": [[[432587.3509999998, 4511685.481699..."
2,2,0,2,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0.0,0.0,0,0,82304,1.684610,0.0,0.573695,"{""rings"": [[[434783.8250000002, 4513368.1227],..."
3,3,0,3,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,2,0,0,0.0,0.0,0,0,82306,1.391100,0.0,0.478397,"{""rings"": [[[436523.17530000024, 4513990.27910..."
4,4,0,4,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,2,0,0,0.0,0.0,0,0,82310,2.187597,0.0,0.381358,"{""rings"": [[[438170.0756000001, 4513697.879000..."


### Read in trip tables, summarize, and format

In [14]:
def summarize_zones(trips_df, name):
    
    # summarize trips by attraction or production
    trips_sum_attr = pd.DataFrame(trips_df.groupby('azone')['trips'].sum())
    trips_sum_prod = pd.DataFrame(trips_df.groupby('pzone')['trips'].sum())
    
    # format tables
    trips_sum_attr['zone_id'] = trips_sum_attr.index
    trips_sum_attr.columns = [name + '_abk', 'zone_id']
    trips_sum_prod['zone_id'] = trips_sum_prod.index
    trips_sum_prod.columns = [name + '_pbk', 'zone_id']
    
    # join the attraction and production summary tables using zone id
    merged = trips_sum_attr.merge(trips_sum_prod, left_on='zone_id', right_on='zone_id', how='outer')
    return merged
    

In [15]:
# sch_univ = pd.read_csv(r".\Data\sch_univ_trip.csv")
# sch_univ_sum = summarize_zones(sch_univ, 'univ')
# sch_univ_sum.isnull().values.any()

In [16]:
# Discretionary trips (social trips, some recreation)
disc = pd.read_csv(r".\Model_Outputs\disc_trip.csv")
disc_sum = summarize_zones(disc, 'disc')
del disc

# Maintenance trips (e.g. groceries)
maint = pd.read_csv(r".\Model_Outputs\maint_trip.csv")
maint_sum = summarize_zones(maint, 'mnt')
del maint

# Maintenance trips non-home-based (e.g. groceries)
maint_nhb = pd.read_csv(r".\Model_Outputs\maint_nhb_trip.csv")
maint_nhb_sum = summarize_zones(maint_nhb, 'mntnhb')
del maint_nhb

# Recreational family trips
rec_fam = pd.read_csv(r".\Model_Outputs\rec_fam_trip.csv")
rec_fam_sum = summarize_zones(rec_fam, 'recfam')
del rec_fam

# Recreation long trips
rec_long = pd.read_csv(r".\Model_Outputs\rec_long_trip.csv")
rec_long_sum = summarize_zones(rec_long, 'reclng')
del rec_long

# Recreation mountain bike trips
rec_mtb = pd.read_csv(r".\Model_Outputs\rec_mtb_trip.csv")
rec_mtb_sum = summarize_zones(rec_mtb, 'recmtb')
del rec_mtb

# Recreation other trips (recreation that doesn't fall into family or long)
rec_oth = pd.read_csv(r".\Model_Outputs\rec_oth_trip.csv")
rec_oth_sum = summarize_zones(rec_oth, 'recoth')
del rec_oth

# school (grade) trips
sch_grade = pd.read_csv(r".\Model_Outputs\sch_grade_trip.csv")
sch_grade_sum = summarize_zones(sch_grade, 'grade')
del sch_grade

# school (university) trips
sch_univ = pd.read_csv(r".\Model_Outputs\sch_univ_trip.csv")
sch_univ_sum = summarize_zones(sch_univ, 'univ')
del sch_univ

# Work trips
work = pd.read_csv(r".\Model_Outputs\work_trip.csv")
work_sum = summarize_zones(work, 'wrk')
del work

# Work non-home-based trips
work_nhb = pd.read_csv(r".\Model_Outputs\work_nhb_trip.csv")
work_nhb_sum = summarize_zones(work_nhb, 'wrknhb')
del work_nhb

In [17]:
rec_fam_sum

,recfam_abk,zone_id,recfam_pbk
0,4.116147,45,NaN
1,0.066573,48,0.018470
2,0.062719,49,0.055407
3,0.857772,51,0.064668
4,0.009304,52,0.037130
...,...,...,...
16906,0.039411,19211,0.162914
16907,0.467984,19212,1.437210
16908,0.262843,19213,1.634462
16909,0.782285,19214,2.480592


In [18]:
zones

,FID,Id,zone_id,CO_TAZID,TAZID,CO_FIPS,CO_NAME,residentia,households,population,jobs1,jobs3,jobs4,jobs5,jobs6,jobs7,jobs9,jobs10,jobs_total,AVGINCOME,ENROL_ELEM,ENROL_MIDL,ENROL_HIGH,HHSIZE_LC1,HHSIZE_LC2,HHSIZE_LC3,PCT_POPLC1,PCT_POPLC2,PCT_POPLC3,PCT_AG1,PCT_AG2,PCT_AG3,INC1,INC2,INC3,INC4,PARK_SCORE,PARK_AREA,SCHOOL_CD,COLL_ENROL,TH_SCORE,COMM_RAIL,LIGHT_RAIL,GQU_RATIO,MIXED_USE,bike_share,industrial,NODE_ID,bklane_len,bkpath_len,AREA_SQMIL,SHAPE
0,0,0,0,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,1,0,0,0.0,0.000000,0,0,75772,0.753915,0.000000,0.037866,"{""rings"": [[[432271.91339999996, 4511646.3509]..."
1,1,0,1,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,1,0,0,0.0,0.000000,0,0,76508,2.480609,0.000000,0.637239,"{""rings"": [[[432587.3509999998, 4511685.481699..."
2,2,0,2,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,1,0,0,0.0,0.000000,0,0,82304,1.684610,0.000000,0.573695,"{""rings"": [[[434783.8250000002, 4513368.1227],..."
3,3,0,3,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,2,0,0,0.0,0.000000,0,0,82306,1.391100,0.000000,0.478397,"{""rings"": [[[436523.17530000024, 4513990.27910..."
4,4,0,4,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,2,0,0,0.0,0.000000,0,0,82310,2.187597,0.000000,0.381358,"{""rings"": [[[438170.0756000001, 4513697.879000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19209,19209,0,19214,570177,177,57,WEBER,0.0,262.0,783.0,129.0,86.0,4.0,471.0,52.0,132.0,21.0,133.0,874.0,72350.640604,0.000538,0.0,0.0,1.880701,4.456423,1.859552,0.217749,0.651696,0.130554,0.316169,0.562229,0.121602,0.117415,0.189279,0.326530,0.366777,0,0,0,0,0,0,0,0.0,201.573944,0,1,109279,2.061338,0.000000,1.353920,"{""rings"": [[[414111.5, 4576365.4], [414166.599..."
19210,19210,0,19215,30131,131,3,BOX ELDER,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53364.818182,0.000000,0.0,0.0,1.674617,3.826075,2.221155,0.166884,0.746178,0.086938,0.286888,0.626268,0.086844,0.226631,0.236271,0.304724,0.232373,2,122819,0,0,0,0,0,0.0,0.000000,0,0,110465,0.000000,0.000000,2.758790,"{""rings"": [[[412799.72869999986, 4580230.78240..."
19211,19211,0,19216,30122,122,3,BOX ELDER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53211.000000,0.000000,0.0,0.0,1.487271,3.322740,1.914478,0.193371,0.707086,0.099543,0.321376,0.579081,0.099543,0.235062,0.239343,0.302038,0.223557,0,0,0,0,0,0,0,0.0,0.000000,0,0,111214,0.114788,0.159798,1.822950,"{""rings"": [[[412913.78000000026, 4586186.91], ..."
19212,19212,0,19217,30059,59,3,BOX ELDER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54392.864222,0.000000,0.0,0.0,0.342818,0.772383,0.444253,0.042962,0.924093,0.032945,0.067375,0.899679,0.032945,0.368686,0.267065,0.236036,0.128213,0,0,1,0,0,0,0,0.0,0.000000,0,0,111109,0.000000,0.000000,1.716000,"{""rings"": [[[414139.03000000026, 4595085.74], ..."


### Merge trip summaries back to microzone shapefile

In [19]:
# Create a clean copy of zones dataset
# zones2 = zones[['zone_id', 'co_tazid', 'tazid', 'co_fips', 'co_name', 'SHAPE']].copy()
zones2 = zones[['zone_id', 'CO_TAZID', 'TAZID', 'CO_FIPS', 'CO_NAME', 'AREA_SQMIL', 'SHAPE']].copy()
zones2['zone_id'] = zones2['zone_id'].astype('int64')



# Join trip tables
zones2 = zones2.merge(disc_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(maint_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(maint_nhb_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(rec_fam_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(rec_long_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(rec_mtb_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(rec_oth_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(sch_grade_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(sch_univ_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(work_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(work_nhb_sum, left_on='zone_id', right_on='zone_id', how='left')




# preview table
zones2.head(5)

,zone_id,CO_TAZID,TAZID,CO_FIPS,CO_NAME,AREA_SQMIL,SHAPE,disc_abk,disc_pbk,mnt_abk,mnt_pbk,mntnhb_abk,mntnhb_pbk,recfam_abk,recfam_pbk,reclng_abk,reclng_pbk,recmtb_abk,recmtb_pbk,recoth_abk,recoth_pbk,grade_abk,grade_pbk,univ_abk,univ_pbk,wrk_abk,wrk_pbk,wrknhb_abk,wrknhb_pbk
0,0,0,0,0,,0.037866,"{'rings': [[[432271.91339999996, 4511646.3509]...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192.346138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,0,0,,0.637239,"{'rings': [[[432587.3509999998, 4511685.481699...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202.976665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,0,0,,0.573695,"{'rings': [[[434783.8250000002, 4513368.1227],...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183.704822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0,0,0,,0.478397,"{'rings': [[[436523.17530000024, 4513990.27910...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,291.655934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0,0,0,,0.381358,"{'rings': [[[438170.0756000001, 4513697.879000...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
zones2.columns

Index(['zone_id', 'CO_TAZID', 'TAZID', 'CO_FIPS', 'CO_NAME', 'AREA_SQMIL',
       'SHAPE', 'disc_abk', 'disc_pbk', 'mnt_abk', 'mnt_pbk', 'mntnhb_abk',
       'mntnhb_pbk', 'recfam_abk', 'recfam_pbk', 'reclng_abk', 'reclng_pbk',
       'recmtb_abk', 'recmtb_pbk', 'recoth_abk', 'recoth_pbk', 'grade_abk',
       'grade_pbk', 'univ_abk', 'univ_pbk', 'wrk_abk', 'wrk_pbk', 'wrknhb_abk',
       'wrknhb_pbk'],
      dtype='object')

In [21]:
# fill na's with 0
zones3 = zones2

# fill NAs where necessary
for field in list(zones3.columns):
    if field not in ['SHAPE']:
        zones3[field].fillna(0, inplace=True)

# calc totals        
zones3['total_abk'] = (zones3['disc_abk'] + zones3['mnt_abk'] + zones3['mntnhb_abk'] + 
                       zones3['recfam_abk'] + zones3['reclng_abk'] + zones3['recmtb_abk'] + zones3['recoth_abk'] + 
                       zones3['grade_abk'] + zones3['univ_abk'] + zones3['wrk_abk'] + zones3['wrknhb_abk']) 

zones3['total_pbk'] = (zones3['disc_pbk'] + zones3['mnt_pbk'] + zones3['mntnhb_pbk'] + 
                       zones3['recfam_pbk'] + zones3['reclng_pbk'] + zones3['recmtb_pbk'] + zones3['recoth_pbk'] + 
                       zones3['grade_pbk'] + zones3['univ_pbk'] + zones3['wrk_pbk'] + zones3['wrknhb_pbk']) 



In [22]:
# then export to shape
zones3.spatial.to_featureclass(location=r".\Outputs\Microzone_Trip_Summaries.shp")

'E:\\Projects\\igraph\\Post_Process_Bike_Model_Outputs\\Outputs\\Microzone_Trip_Summaries.shp'

## Merge zone attraction and production scores with the microzone geometry

In [23]:
# Create a clean copy of zones dataset
# zones2 = zones[['zone_id', 'co_tazid', 'tazid', 'co_fips', 'co_name', 'SHAPE']].copy()
zones2 = zones[['zone_id', 'CO_TAZID', 'TAZID', 'CO_FIPS', 'CO_NAME', 'AREA_SQMIL', 'SHAPE']].copy()
zones2['zone_id'] = zones2['zone_id'].astype('int64')

# NOTE: need to add zone_id to empty field in output csv
ascore = pd.read_csv(r".\Model_Outputs\zone_attraction_size.csv")
pscore = pd.read_csv(r".\Model_Outputs\zone_production_size.csv")

zones3a = zones2.merge(ascore, left_on='zone_id', right_on='zone_id', how='left')
zones3p = zones2.merge(pscore, left_on='zone_id', right_on='zone_id', how='left')

In [24]:
# fill NAs where necessary
for field in list(zones3a.columns):
    if field !='SHAPE':
        zones3a[field].fillna(-1, inplace=True)

# fill NAs where necessary
for field in list(zones3p.columns):
    if field !='SHAPE':
        zones3p[field].fillna(-1, inplace=True)

zones3p.rename({'sch_grade_nhb': 'grade_nhb', 'sch_univ_nhb': 'univ_nhb', 'rec_oth_nhb':'recothnhb'}, axis=1, inplace=True)        
        
# Fill NAs with -1, then export to shape
zones3a.spatial.to_featureclass(location=r".\Outputs\Microzone_A_Scores.shp")
zones3p.spatial.to_featureclass(location=r".\Outputs\Microzone_P_Scores.shp")

'E:\\Projects\\igraph\\Post_Process_Bike_Model_Outputs\\Outputs\\Microzone_P_Scores.shp'

## Get Centroid Nodes

In [25]:
nodes = pd.DataFrame.spatial.from_featureclass(r".\Model_Outputs\nodes.shp")
nodes['node_id'] = nodes.index
nodes.shape

(112170, 7)

In [26]:
nodes2 = nodes[['node_id', 'xcoord', 'ycoord', 'zcoord', 'SHAPE']].copy()
centroids = nodes2.merge(zones[['NODE_ID', 'zone_id']], left_on='node_id', right_on='NODE_ID', how='inner')
print(centroids.columns)

Index(['node_id', 'xcoord', 'ycoord', 'zcoord', 'SHAPE', 'NODE_ID', 'zone_id'], dtype='object')


In [27]:
centroids = centroids[['node_id', 'xcoord', 'ycoord', 'zcoord', 'zone_id', 'SHAPE']].copy()
centroids.spatial.to_featureclass(location=r".\Outputs\Microzone_Centroids.shp")

'E:\\Projects\\igraph\\Post_Process_Bike_Model_Outputs\\Outputs\\Microzone_Centroids.shp'